# Import

In [ ]:
from pandas import read_csv
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

url="https://raw.githubusercontent.com/jbrownlee/Datasets/master/iris.csv"
names=['sepal-length','sepal-width','petal-length','petal-width','class']
df=read_csv(url,names=names)
df_uni = df[ df["class"] != 'Iris-virginica']
le = preprocessing.LabelEncoder()
le.fit(list(df_uni["class"].unique()))
df_uni["class"] = le.transform(df_uni["class"])
print(df_uni.shape)

In [ ]:
X=df_uni.iloc[:,:2]
y=df_uni.iloc[:,df_uni.shape[1]-1]


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Sigmoid

In [ ]:
def sig(arr):
    return 1/ (1+np.exp(-np.array(arr)))

# Forward

In [ ]:
def forward (W,B,row):
    a = row
    A = []
    H = []
    for i in range(len(W)):
        h = np.dot(a,W[i])
        r = h+B[i].T[0]
        a = sig(r)
        H.append(h)
        A.append(a)
    return A , H

# Backward

In [ ]:
def backward(n,A,H,w,B,lr):
    inp = X_train.iloc[n].to_list()
    A_b = A.copy()
    A_b.insert(0,np.array(inp))
    O=A_b[len(A_b)-1]
    A_b.pop()
    
    ################ intialize
    devw = list(np.ones(np.shape(dev[:len(dev)-1])))
    devw.append(np.ones((len(dev[len(dev)-1]),1)))
    
    devb = list(np.ones(np.shape(B[:len(B)-1])))
    devb.append(np.ones((len(B[len(B)-1]),1)))

    #################### output shape
    if( np.shape(W[ len(W)-1 ])[1] == 2 ): # for two outputs
        if(y_train.iloc[n] == 0): 
            T = [1 , 0]
        else:
            T = [0,1]
   
    elif(  np.shape(W[ len(W)-1 ])[1] == 1 ): # for one output
        T = y_train.iloc[n]  
    
    ###################### constant term
    f = len(w)-1
    err = np.sum((T-O)**2/2)
    #print("error in batch",n,"  :  " ,err)
    const = -2*(T-O) * sig(H[f])*(1-sig(H[f]))
    
    ###################  main update
    for i in range(f+1):
        j=i
        devw[i] = np.dot(const.reshape(-1,1) , A_b[i].reshape(-1,1).T)
        devb[i] = const.reshape(-1,1)
        
        while( j < f):
            devw[i] = devw[i]  *  w[j+1] * sig(H[j])*(1-sig(H[j]))
            devb[i] = devb[i]  *  np.sum( W[j+1] * sig(H[j])*(1-sig(H[j])) )
            j+=1
        
        if( np.array(w[i]).shape !=(2,2) ):
            for s in range(len(w[i])):
                w[i][s] = w[i][s]-lr*devw[i][0][s]
                B[i] = B[i] - lr*devb[i]
        
        else:
            w[i] = w[i] - lr*devw[i]
            B[i] = B[i] - lr*devb[i]
       
    return w,B,err

# Intialize

### reshape shape to take any input layers length by increasing its length and changing its numbers to get any number of nodes & adjust your T in backward function easily by adapting to your number of output - add  1 or 0 - and choose if you want momentum

In [ ]:
shape = [2,2,1] # [,,x] -> (len(shape)-1) layers & x output
W = []
dev = []
B=[]
for i in range(len(shape)-1):
    l = np.random.rand(shape[i],shape[i+1])
    W.append(l)
    l_b = np.random.rand(shape[i+1],1)
    B.append(l_b)
    dev.append(l)
W_t = W
B_t = B
lr = lr1 = lr2 = 0.01
momentum = False

# Fit

In [ ]:
for i in range(10000): # epoch
    
    X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.3)
    
    for i in range(X_train2.shape[0]-1): #batch
        row = X_train2.iloc[i,:]
        As,Hs = forward(W,B,row)
        
        W_old = W
        W,B,err = backward(i,As,Hs,W,B,lr)
        W_new=W
        
        if(momentum):
            for mom in range(len(W)):
                W[mom] = lr1 * W_old[mom] + lr2 * W_new[mom]
                
        if(err<0.0001):
            break
    #######################
    for i in range(len(W)):
        W[i] = W[i]/np.max(W[i])
        B[i] = B[i]/np.max(B[i])
        
    O_l=[]
    for i in range(X_test2.shape[0]): #validate to choose best split
        row = X_test2.iloc[i,:]
        As,Hs = forward(W,B,row)
        O=As[len(As)-1]
        O_l.append(O[0])
    m = np.mean(O_l)-lr

# Test

In [ ]:
for i in range(y_test.shape[0]):
    row = X_test.iloc[i,:]
    As,Hs = forward(W,B,row)
    O=As[len(As)-1]
    if(O < m):
        O_compare = 0
    else:
        O_compare = 1
    if(O_compare!=y_test.iloc[i]):
        err+=1
        
print(err/y_test.shape[0]*100)